# Port Performance Index Project - Main Notebook

This notebook presents the primary data analysis related to the PPI Project; see the README and other files in the [repo](https://github.com/epistemetrica/Port-Performance-Project) for full details. 

## Data Processing

AIS data is ingested from the Marine Cadastre database via the scripts found in the vessel data folder. Here, we combine the AIS data with port-level data and prepare for analysis. 

In [1]:
#prelims
import polars as pl
import pandas as pd
import geopandas as gpd
import time
import os

#enable string cache for polars categoricals
pl.enable_string_cache()
#display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pl.Config(tbl_rows=200);

### AIS Data

The vessel locations and status (e.g., "under way", "anchored", "moored") data include all AIS messages; for the purposes of the PPI, we only need to know when a vessel *changes* status, so all other observations are dropped. 

We also limit the data for the PPI to vessels over 100m in length. 

In [2]:
%%script echo skipping

ais_gdf = (
    #read into lazyframe
    pl.scan_parquet('ais data/data/ais_clean/*.parquet')
    #drop smaller vessels
    .filter(pl.col('length')>100)
    #sort by vessel and time
    .sort(['mmsi', 'time'])
    #indicate whether status is the same as previous row (Fill value needed to avoid status 0 evaluating as equal to false)
    .with_columns(
        status_change = (
            pl.col('status').ne(pl.col('status').shift(fill_value=20))
            .over('mmsi')
        )
    )
    #keep only new status pings
    .filter(pl.col('status_change')==True)
    #drop change col
    .drop('status_change')
    #collect to pandas df
    .collect()
    .to_pandas()
)

#convert to geopandas dataframe
ais_gdf = (
    #convert to geodataframe
    gpd.GeoDataFrame(
        ais_gdf,
        geometry=gpd.points_from_xy(ais_gdf.lon, ais_gdf.lat, crs='EPSG:4326')
    )
    #convert to WGS84 pseudo-mercator
    .to_crs(3857)
    #drop old lat lon cols
    .drop(['lat', 'lon'], axis=1)
)

#write to parquet
ais_gdf.to_parquet('ais data/data/ais_status_changes.parquet')

skipping


In [4]:
#read previously processed AIS parquet file
ais_gdf = gpd.read_parquet('ais data/data/ais_status_changes.parquet')

### Port and Dock Data

Locations and descriptions for each dock and port come from the BTS and USACE online databases. 

In [3]:
#load port data
ports_gdf = (
    #read in shape file downloaded from BTS
    gpd.read_file('port data/Principal_Ports/Principal_Ports.shp')
    #drop unneeded columns
    .drop([
        'FID', #randomly assigned table id
        'PORT', #unknown numeric ID - not CBP or UN code
        'FOREIGN_','EXPORTS', 'IMPORTS', 'DOMESTIC' #breadown of total vol (tons)
    ], axis=1)
)
#set col names to pythonic lowercase
ports_gdf.columns = ports_gdf.columns.str.lower()

#load dock data
docks_gdf = (
    #read in shape file downloaded from USACE
    gpd.read_file('port data/Dock/Dock.shp')
    #drop unneeded columns
    .drop([
        'FID', #randomly assigned table id
        'LONGITUDE', 'LATITUDE', #already coded in 'geometry' 
        'LOCATION_D', #text description of dock location
        'STREET_ADD','ZIPCODE', #street address details
        'PSA_NAME', #statistical area name, rarely used
        'COUNTY_NAM', 'COUNTY_FIP', 'CONGRESS', 'CONGRESS_F', #county and congress info
        'MILE', 'BANK', 'LATITUDE1', 'LONGITUDE1', #redundant locaation data
        'OPERATORS', 'OWNERS', #owner info
        'PURPOSE', #long-form text description of dock uses
        'DOCK', #unknown number (not unique to each row/dock)
        'HIGHWAY_NO', 'RAILWAY_NO', 'LOCATION', #redundant location info
        'COMMODITIE', 'CONSTRUCTI','MECHANICAL', 'REMARKS', 'VERTICAL_D', 
        'DEPTH_MIN', 'DEPTH_MAX','BERTHING_L', 'BERTHING_T', 'DECK_HEIGH', 
        'DECK_HEI_1', #these are rarely used stats on construction
        'SERVICE_IN','SERVICE_TE', #rarely used indicators of data entry date 
    ], axis=1)
    #rename cols for clarity
    .rename(columns={
        'NAV_UNIT_I':'nav_unit_id',
        'NAV_UNIT_N':'nav_unit_name',
        'FACILITY_T':'facility_type',
        'CITY_OR_TO':'city',
        'STATE_POST':'state'
    })
)
#set col names to pythonic lowercase
docks_gdf.columns = docks_gdf.columns.str.lower()

### Matching

We first match each of the anchored and moored points from the AIS data with the nearest Port. 

In [4]:
start = time.time()
stops_gdf = (
    #filter to only moorings
    ais_gdf[ais_gdf.status == 5]
    #join in nearest port to each ais message
    .sjoin_nearest(ports_gdf, how='left', exclusive=True)
    #drop unneeded cols
    .drop(['index_right', 'total'], axis=1)
    #rename cols for clarity
    .rename({'rank':'port_rank', 'type':'port_type'}, axis=1)
)
print(f'Spatial Join time on {len(stops_gdf)} AIS messages took {time.time()-start} seconds')

#create main df
main_gdf = (
    #merge stops back into AIS data
    ais_gdf.merge(stops_gdf, how='left')
    #sort by vessel then time of message
    .sort_values(by=['mmsi', 'time'])
)
#backfill port info except geometry (normal pandas syntax not supported for gpd geometry)
main_gdf[['port_type','port_name','port_rank']] = (
    main_gdf[['port_type','port_name','port_rank']].bfill()
)
#merge port geometries into main (NOTE backfill not supported for gpd geometry, hence the separate merge step)
main_gdf = main_gdf.merge(ports_gdf[['port_name', 'geometry']], 
                          on='port_name', how='left', suffixes=[None, '_port'])
#compute distance from message loc to port loc
main_gdf['port_dist'] = main_gdf['geometry'].distance(main_gdf['geometry_port'])

Spatial Join time on 187873 AIS messages took 0.6062629222869873 seconds


In [5]:
#inspect results
main_gdf.head()

,mmsi,time,speed,course,heading,status,vessel_name,vessel_type,imo,length,width,draft,cargo,geometry,port_type,port_name,port_rank,geometry_port,port_dist
0,3,2017-12-17 02:27:29,0.0,60.0,35.0,1.0,MAERSK PRIVILEGE,80.0,3.0,240.0,42.0,-12.6,80.0,POINT (-10021529.366 3474701.894),C,"PortMiami, FL",52.0,POINT (-8925779.558 2972212.233),1.205472e+06
1,3,2017-12-17 03:34:09,1.0,46.0,41.0,0.0,MAERSK PRIVILEGE,80.0,3.0,240.0,42.0,-12.6,80.0,POINT (-10021509.329 3474712.155),C,"PortMiami, FL",52.0,POINT (-8925779.558 2972212.233),1.205458e+06
2,36336,2019-10-28 10:45:42,12.5,46.0,49.0,0.0,NAVIOS LYRA,79.0,9498626.0,231.0,39.0,5.8,79.0,POINT (-8231327.164 4246784.482),C,"PortMiami, FL",52.0,POINT (-8925779.558 2972212.233),1.451482e+06
3,102810,2015-07-26 02:13:42,0.0,13.6,14.0,0.0,SINGLEVESSEL,79.0,4711.0,130.0,16.0,0.0,79.0,POINT (-13135627.556 3988362.673),C,"PortMiami, FL",52.0,POINT (-8925779.558 2972212.233),4.330748e+06
4,21059000,2017-03-14 02:33:03,0.0,0.0,228.0,5.0,SANTIAGO,70.0,9255050.0,172.0,27.0,10.5,70.0,POINT (-8338607.984 2350996.438),C,"PortMiami, FL",52.0,POINT (-8925779.558 2972212.233),8.547979e+05


In [6]:
df = (
    pl.DataFrame(main_gdf.drop(['geometry', 'geometry_port'], axis=1))
    .with_columns(month = pl.col('time').dt.strftime('%Y%m'))
)

df.head()

mmsi,time,speed,course,heading,status,vessel_name,vessel_type,imo,length,width,draft,cargo,port_type,port_name,port_rank,port_dist,month
i64,datetime[μs],f64,f64,f64,f64,cat,f64,f64,f64,f64,f64,f64,str,str,f64,f64,str
3,2017-12-17 02:27:29,0.0,60.0,35.0,1.0,"""MAERSK PRIVILEGE""",80.0,3.0,240.0,42.0,-12.6,80.0,"""C""","""PortMiami, FL""",52.0,1.2055e6,"""201712"""
3,2017-12-17 03:34:09,1.0,46.0,41.0,0.0,"""MAERSK PRIVILEGE""",80.0,3.0,240.0,42.0,-12.6,80.0,"""C""","""PortMiami, FL""",52.0,1.2055e6,"""201712"""
36336,2019-10-28 10:45:42,12.5,46.0,49.0,0.0,"""NAVIOS LYRA""",79.0,9.498626e6,231.0,39.0,5.8,79.0,"""C""","""PortMiami, FL""",52.0,1.4515e6,"""201910"""
102810,2015-07-26 02:13:42,0.0,13.6,14.0,0.0,"""SINGLEVESSEL""",79.0,4711.0,130.0,16.0,0.0,79.0,"""C""","""PortMiami, FL""",52.0,4.3307e6,"""201507"""
21059000,2017-03-14 02:33:03,0.0,0.0,228.0,5.0,"""SANTIAGO""",70.0,9.25505e6,172.0,27.0,10.5,70.0,"""C""","""PortMiami, FL""",52.0,854797.94178,"""201703"""
